Este programa se encarga de obtener los usuarios conectados a un determinado AP en un rango de fechas establecido.

Creamos 2 programas:
- csv_tool: se encarga de extraer la informacion y aplicar expresiones regulares
- app_cli: se encarga de filtar la informacion con la MACAddress del AP elegido y con las fechas provistas por el usuario

csv_tool

Primero lo que haces es definir las expresiones regulares que vamos a usar

In [4]:
import re

MAC_RE = re.compile(r'([0-9A-Fa-f]{2}[:-]){5}([0-9A-Fa-f]{2}):HCDD$')
DATE_RE = re.compile(r'(20(1[5-9]|2[0-5])[-/](0[1-9]|1[0-2])[-/]([0-2][0-9]|3[0-1])$)')
USER_RE = re.compile(r'[a-zA-Z0-9-_\\/]{1,25}$')

Luego tiene 2 funciones, donde la primera extrae la informacion del csv (create_pandas) y la otra aplica las expresiones regulares (apply_regex)

In [5]:
import pandas as pd

def create_pandas(csv_path: str, columns: list = ["MAC_AP", "Inicio_de_Conexión_Dia", "FIN_de_Conexión_Dia", "Usuario"]):
    data = pd.read_csv(csv_path,
                       low_memory=False,
                       usecols=columns)

    return data


def apply_regex(data: pd.DataFrame):
    mask = (data["MAC_AP"].str.fullmatch(MAC_RE) &
            data["Inicio_de_Conexión_Dia"].str.fullmatch(DATE_RE) &
            data["FIN_de_Conexión_Dia"].str.fullmatch(DATE_RE) &
            data["Usuario"].str.fullmatch(USER_RE))
    data = data[mask]

    return data

app_cli:

El cuerpo del codigo es una sola funcion (menu), que se encarga de todo lo dicho al principio de este documento. Esta funcion tiene 1 parametro que es el path del csv

Primero lo que hacemos es llamar a las funciones create_pandas y apply_regex, ademas creamos unos spinner para que el usuario sepa que tareas esta llevando a cabo el programa

In [ ]:
import typer
from yaspin import yaspin
from csv_tool import create_pandas, apply_regex, DATE_RE

def menu(csv_path: str = typer.Option(..., prompt="Ingrese la ruta del archivo csv")):
    csv_read_spinner = yaspin(text="Leyendo csv", color="yellow")
    apply_regex_spinner = yaspin(text="Aplicando expresiones regulares", color="yellow")

    csv_read_spinner.start()
    data = create_pandas(csv_path)
    csv_read_spinner.stop()

    apply_regex_spinner.start()
    data = apply_regex(data)
    apply_regex_spinner.stop()

Luego, creamos una lista de las MACAddress de los APs para que el usuario seleccione uno, ademas de pedirle el rango de fechas

In [ ]:
import typer
import inquirer
import pandas as pd


def menu(csv_path: str = typer.Option(..., prompt="Ingrese la ruta del archivo csv")):
    data = create_pandas(csv_path)
    data = apply_regex(data)
    
    ap_list = data["MAC_AP"].unique().tolist()

    ap = inquirer.prompt([inquirer.List('ACCESS POINT', message="Seleccione un AP", choices=ap_list)])

    fechas = inquirer.prompt([
        inquirer.Text("FECHA DE INICIO", message="Ingrese la fecha de inicio (YYYY/MM/DD)", validate=lambda _, x: DATE_RE.fullmatch(x) is not None),
        inquirer.Text("FECHA DE FIN", message="Ingrese la fecha de fin (YYYY/MM/DD)", validate=lambda _, x: DATE_RE.fullmatch(x) is not None),
    ])

    ap, fecha_i, fecha_f = ap["ACCESS POINT"], fechas["FECHA DE INICIO"], fechas["FECHA DE FIN"]

    fecha_i = pd.to_datetime(fecha_i)
    fecha_f = pd.to_datetime(fecha_f)
    data["Inicio_de_Conexión_Dia"] = pd.to_datetime(data["Inicio_de_Conexión_Dia"])
    data["FIN_de_Conexión_Dia"] = pd.to_datetime(data["FIN_de_Conexión_Dia"])

Mas adelante, filtramos el DataFrame con la informacion proporcionada por el usuario

In [ ]:
def menu(csv_path: str = typer.Option(..., prompt="Ingrese la ruta del archivo csv")):

    filtro = (
        (data["MAC_AP"] == ap)
        & (data["Inicio_de_Conexión_Dia"].dt.year >= fecha_i.year)
        & (data["Inicio_de_Conexión_Dia"].dt.month >= fecha_i.month)
        & (data["Inicio_de_Conexión_Dia"].dt.day >= fecha_i.day)
        & (data["FIN_de_Conexión_Dia"].dt.year <= fecha_f.year)
        & (data["FIN_de_Conexión_Dia"].dt.month <= fecha_f.month)
        & (data["FIN_de_Conexión_Dia"].dt.day <= fecha_f.day)
    )

    data = data[filtro].groupby(["Usuario"]).count().reset_index()
    users = data["Usuario"].unique()

Por ultimo imprimimos la tabla con los usuarios por pantalla y la guardamos en un archivo csv.

In [ ]:
print(tabulate([[user] for user in users], headers=[f"Usuarios conectados al AP {ap} entre {fecha_i} y {fecha_f}"], tablefmt='grid', stralign='center'))

with open("output.csv", "w") as f:
    f.write(data["Usuario"].to_csv(index=False))